<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (227,227,3))

In [0]:
'''
CLASSES = 2
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
'''


"\nCLASSES = 2\nx = base_model.output\nx = GlobalAveragePooling2D(name='avg_pool')(x)\nx = Dropout(0.4)(x)\npredictions = Dense(CLASSES, activation='softmax')(x)\nmodel = Model(inputs=base_model.input, outputs=predictions)\n"

"\nCLASSES = 2\nx = base_model.output\nx = GlobalAveragePooling2D(name='avg_pool')(x)\nx = Dropout(0.4)(x)\npredictions = Dense(CLASSES, activation='softmax')(x)\nmodel = Model(inputs=base_model.input, outputs=predictions)\n"

In [0]:
base_model.trainable = False

In [0]:
add_model = Sequential()
add_model.add(base_model)
add_model.add(Conv2D(96, kernel_size=5, strides=1))
add_model.add(Activation('relu'))

add_model.add(Conv2D(256, kernel_size=3, strides=1))
add_model.add(Activation('relu'))

#add_model.add(Conv2D(96, kernel_size=3, strides=1))
#add_model.add(Activation('relu'))

#add_model.add(GlobalAveragePooling2D())
#add_model.add(Dropout(0.5))
add_model.add(Flatten())
add_model.add(Dense(2))
add_model.add(Activation('sigmoid'))


model = add_model
kazim = tf.keras.optimizers.Adam(learning_rate=0.001)
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer=kazim,
              metrics=['accuracy'],
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d (Conv2D)              (None, 3, 3, 96)          1228896   
_________________________________________________________________
activation (Activation)      (None, 3, 3, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 256)         221440    
_________________________________________________________________
activation_1 (Activation)    (None, 1, 1, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 5

In [0]:
train_data_dir = '/content/drive/My Drive/Augmented '

In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
batch_size = 32
img_width, img_height = 227, 227
# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 7716 images belonging to 2 classes.
Found 1929 images belonging to 2 classes.
Found 7716 images belonging to 2 classes.
Found 1929 images belonging to 2 classes.


In [0]:
epochs = 100
train_samples = 7716
validation_samples = 1929

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/100
32/32 [==============================] - 966s 30s/step - loss: 0.6109 - acc: 0.6738

Epoch 00001: val_acc improved from -inf to 0.67383, saving model to Breast.h5
32/32 [==============================] - 1885s 59s/step - loss: 0.6891 - acc: 0.5742 - val_loss: 0.6109 - val_acc: 0.6738
Epoch 2/100
32/32 [==============================] - 517s 16s/step - loss: 0.6172 - acc: 0.6631

Epoch 00002: val_acc did not improve from 0.67383
32/32 [==============================] - 1136s 36s/step - loss: 0.6194 - acc: 0.6621 - val_loss: 0.6172 - val_acc: 0.6631
Epoch 3/100
32/32 [==============================] - 502s 16s/step - loss: 0.5950 - acc: 0.6719

Epoch 00003: val_acc did not improve from 0.67383
32/32 [==============================] - 1101s 34s/step - loss: 0.6269 - acc: 0.6436 - val_loss: 0.5950 - val_acc: 0.6719
Epoch 4/100
32/32 [==============================] - 517s 16s/step - loss: 0.5788 - acc: 0.7017

Epoch 00004: val_acc improved from 0.67383 to 0.70166, saving model 